In [1]:
import altair as alt
import pandas as pd

from pathlib import Path
from theme import theme

In [2]:
alt.themes.register("latex", theme)
alt.themes.enable("latex")

ThemeRegistry.enable('latex')

In [3]:
name = "position-bias"
directory = Path("figures/")

## Figure 2: Position bias estimates on Baidu ULTR

In [4]:
method2name = {
    "ctr": "CTR",
    "pivot_one": "Pivot One",
    "adjacent_chain": "Adjacent Chain",
    "global_all_pairs": "All Pairs",
}

In [5]:
dfs = []
path = Path("../propensities")

for file in path.glob("*.csv"):
    df = pd.read_csv(file)

    method = df.columns[1]
    df["name"] = method2name[method]
    df = df.rename(columns={method: "examination"})
    dfs.append(df)

df = pd.concat(dfs)
df.head()

,position,examination,name
0,1,1.000000,All Pairs
1,2,0.673779,All Pairs
2,3,0.414474,All Pairs
3,4,0.293205,All Pairs
4,5,0.207862,All Pairs


In [6]:
df = df[df["position"] <= 10]
ctr_df = df[df["name"] == "CTR"]
harvesting_df = df[df["name"] != "CTR"]
order = list(method2name.values())

In [7]:
ctr = alt.Chart(ctr_df, width=400).mark_line(size=2, opacity=1, point=True, strokeDash=[8,2]).encode(
    x=alt.X("position:Q", title="Position").scale(domain=(1, 10)),
    y=alt.Y("examination", title="Examination"),
    color=alt.Color("name"),
)

harvesting = alt.Chart(harvesting_df, width=400).mark_line(size=3, opacity=0.9, point=True).encode(
    x=alt.X("position:Q", title="Position").scale(domain=(1, 10)),
    y=alt.Y("examination", title="Examination"),
    color=alt.Color("name", title=None, sort=order).scale(domain=order),
)

chart = (ctr + harvesting).configure_point(
    size=60
).configure_legend(
    orient="none",
    direction="horizontal",
    legendX=17, legendY=-25,

)

chart

alt.LayerChart(...)

In [8]:
svg_file = directory / f"{name}.svg"
pdf_file = directory / f"{name}.pdf"

In [9]:
# to .svg
chart.save(svg_file)
# to .pdf
!rsvg-convert -f pdf {svg_file} > {pdf_file}